In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/whodunnit/test_tweets_unlabeled.txt
/kaggle/input/whodunnit/train_tweets/train_tweets.txt
/kaggle/input/dataset1/test_tweets_unlabeled.csv
/kaggle/input/dataset1/train_tweets.csv
/kaggle/input/dataset/test_tweets_unlabeled.csv
/kaggle/input/dataset/train_tweets.csv


In [2]:
#Major imports
from sklearn import metrics
from keras.layers import Dense, Embedding, Activation, merge, Input, Lambda, Reshape,BatchNormalization
from keras.layers import Conv1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D,SeparableConv1D
from keras.utils import to_categorical,multi_gpu_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import multiprocessing
from keras.models import Model
from keras import regularizers
from keras.layers.merge import concatenate
import re
import nltk
import glob
from sklearn.cluster import KMeans
from scipy.cluster.vq import whiten
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from multiprocessing import Pool

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [3]:
!ls ../input/
testingData = pd.read_csv('../input/dataset1/test_tweets_unlabeled.csv')
trainingData = pd.read_csv('../input/dataset1/train_tweets.csv')

dataset  dataset1  whodunnit


In [4]:
#majority of the following pre-processes text method was found online
def pp_text(txt):

    HTTP_URL_PATTERN = r'((http|ftp|https):\/\/)?([\w-]+(?:(?:\.[\w-]{2,})+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'
    
    #handles etc
    txt = txt.strip()
    txt = re.sub(HTTP_URL_PATTERN, ' HAVELINK ', txt)
    txt = re.sub(r'@handle', ' ATSOMEBODY ', txt)
    txt = re.sub(r'^RT', ' APURERETWEET ', txt)
    txt = re.sub(r'RT(?!\w)', ' WITHARETWEET ', txt)

    #emphasise hashtag
    current = re.search(r'#(?P<hashtag>\w+?)\b', txt)
    while(current is not None):
        hashtag = current.group('hashtag')
        txt = re.sub(r'#'+hashtag, (' '+hashtag+' ')*1, txt)
        current = re.search(r'#(?P<hashtag>\w+?)\b', txt)

    #specifics
    txt = re.sub('\d+', ' ', txt)
    txt = re.sub('\s+', ' ', txt)
    txt = re.sub('[!@#$_]', ' ', txt)
    txt = re.sub('\s\W', ' ', txt)
    txt = re.sub('\W,\s', ' ', txt)
    txt = re.sub(r'\W', ' ', txt)
    txt = txt.lower()

    #stopwords
    pat = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    txt = pat.sub('', txt)

    lemma = nltk.WordNetLemmatizer()
    new_txt = ""
    for word in txt.split():
        new_txt += lemma.lemmatize(word)
        new_txt += " "
    return(new_txt.strip())

In [5]:
#actually preprocess the data
testingData['processed'] = testingData['tweet'].apply(pp_text)
trainingData['processed'] = trainingData['tweet'].apply(pp_text)

every_author = trainingData['author'].unique().tolist()
i_authors = dict(enumerate(every_author))
author_i = dict((author,i) for i, author in i_authors .items())
trainingData['id'] = trainingData['author'].apply(lambda author: author_i[author])

In [6]:
authors_count = len(i_authors )
train_authors = to_categorical(trainingData['id'],num_classes = authors_count)

In [7]:
#10000
word_count = 30000
limit = 250
tokenizer = Tokenizer(num_words = word_count)
allData = pd.concat([trainingData['processed'],testingData['processed']])
tokenizer.fit_on_texts(allData)
trainingSeries = tokenizer.texts_to_sequences(trainingData['processed'])
currentTrainingData = pad_sequences(trainingSeries, maxlen = limit, padding='post')
testingSeries = tokenizer.texts_to_sequences(testingData['processed'])
currentTestingData = pad_sequences(testingSeries, maxlen = limit, padding='post')

In [8]:
print("--- Training Begin ---")

###Common CNN implementation from internet
def cnn(words_num,embedding_dims,limit,num_class):
    tensor_input = Input(shape=(limit,), dtype='float64')
    embed = Embedding(words_num+1, embedding_dims)(tensor_input)
    cnn1 = SeparableConv1D(200, 3, padding='same', strides = 1, activation='relu',kernel_regularizer = regularizers.l1(0.00001))(embed)
    cnn1 = BatchNormalization()(cnn1)
    cnn1 = MaxPool1D(pool_size = 100)(cnn1)
    cnn2 = SeparableConv1D(200, 4, padding='same', strides = 1, activation='relu',kernel_regularizer = regularizers.l1(0.00001))(embed)
    cnn2 = BatchNormalization()(cnn2)
    cnn2 = MaxPool1D(pool_size = 100)(cnn2)
    cnn3 = SeparableConv1D(200, 5, padding='same', strides = 1, activation='relu',kernel_regularizer = regularizers.l1(0.00001))(embed)
    cnn3 = BatchNormalization()(cnn3)
    cnn3 = MaxPool1D(pool_size = 100)(cnn3)
    cnn = concatenate([cnn1,cnn2,cnn3], axis=-1)
    dropout = Dropout(0.5)(cnn)
    flatten = Flatten()(dropout)
    dense =  Dense(128, activation='relu')(flatten)
    dense = BatchNormalization()(dense)
    dropout = Dropout(0.5)(dense)
    tensor_output = Dense(num_class, activation='softmax')(dropout)
    model = Model(inputs = tensor_input, outputs = tensor_output)
    print(model.summary())
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model_sepcnn = cnn(words_num = len(tokenizer.word_index),embedding_dims = 300,limit = limit,num_class = authors_count)
model_sepcnn.fit(currentTrainingData,train_authors,epochs = 35, batch_size = 512)
print('training done')

pred_ = [model_sepcnn.predict(vec.reshape(1,limit)).argmax() for vec in currentTestingData]
testingData['predicted'] = [i_authors[i] for i in pred_]
metrics.accuracy_score(testingData['id'],testingData['predicted'])


print("--- Training Complete ---")

--- Training Begin ---
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 250)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 250, 300)     35114100    input_1[0][0]                    
__________________________________________________________________________________________________
separable_conv1d_1 (SeparableCo (None, 250, 200)     61100       embedding_1[0][0]                
__________________________________________________________________________________________________
separable_conv1d_2 (SeparableCo (None, 250, 200)     61400       embedding_1[0][0]                
______________________________________________________________________________________

In [9]:
output = pd.DataFrame({'id': range(1, testingData.shape[0] + 1),
                       'predicted': testingData['predicted']})
output.to_csv('./submission-Peng.csv', index = False)

print("output succeed")

output succeed
